In [ ]:
class GeneratorCGAN(nn.Module):
    def __init__(self, nz=100, nc=1, input_size=32, class_num=10):
        super(GeneratorCGAN, self).__init__()
        self.nz = nz
        self.nc = nc
        self.input_size = input_size
        self.class_num = class_num

      
        self.fc = nn.Sequential(
            nn.Linear(self.nz + self.class_num, 1024),
            nn.ReLU(),
            nn.Linear(1024, 128 * (self.input_size // 4) * (self.input_size // 4)),
            nn.ReLU(),
        )
    
        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, self.nc, 4, 2, 1),
            nn.Tanh(),  
        )

        self.apply(self._initialize_weights)

    def forward(self, input, label):
        x = torch.cat([input, label], 1)
        

        x = self.fc(x)
        x = x.view(-1, 128, (self.input_size // 4), (self.input_size // 4))

    
        x = self.deconv(x)

        return x

    def _initialize_weights(self, m):
        if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Linear):
            nn.init.normal_(m.weight, 0, 0.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)


In [ ]:
class DiscriminatorACGAN(nn.Module):

    def __init__(self, nz=1, nc=1, input_size=32, class_num=10):
        super(DiscriminatorACGAN, self).__init__()
        self.nz = nz
        self.nc = nc
        self.input_size = input_size
        self.class_num = class_num

        self.conv = nn.Sequential(
            nn.Conv2d(self.nz, 64, 4, 2, 1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
        )
        self.fc1 = nn.Sequential(
            nn.Linear(128 * (self.input_size // 4) * (self.input_size // 4), 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
        )
        self.dc = nn.Sequential(
            nn.Linear(1024, self.nc),
            nn.Sigmoid(),
        )
        self.cl = nn.Sequential(
            nn.Linear(1024, self.class_num),
        )
        initialize_weights(self)

    def forward(self, input):
        x = self.conv(input)
        x = x.view(-1, 128 * (self.input_size // 4) * (self.input_size // 4))
        x = self.fc1(x)
        d = self.dc(x)
        c = self.cl(x)

        return d, c

In [ ]:
# networks init CGAN-ACGAN

input_size = 28
z_dim = 62
class_num = 10
sample_num = class_num ** 2
batch_size = 64

# networks init
lrG= 0.001
lrD=0.0002
beta1=0.5
beta2=0.99

G = GeneratorCGAN(nz=z_dim, nc=1, input_size=input_size, class_num=class_num)
D = DiscriminatorACGAN(nz=1, nc=1, input_size=input_size, class_num=class_num)

print(G)
print(D)

optimizerG = optim.Adam(G.parameters(), lr=lrG, betas=(beta1, beta2))
optimizerD = optim.Adam(D.parameters(), lr=lrD, betas=(beta1, beta2))

# Define loss

BCE_loss = nn.BCELoss()
CE_loss = nn.CrossEntropyLoss()

In [ ]:
d_losses_cgan_acgan ,g_losses_cgan_acgan , SSIM_SCORES_cgan_acgan ,KID_SCORES_cgan_acgan = train_model("CGAN_ACGAN",G,D,"36")

In [ ]:
plot_losses(d_losses_cgan_acgan, g_losses_cgan_acgan,"CGAN-ACGAN")
ssim_plot(SSIM_SCORES_cgan_acgan,"CGAN-ACGAN")
kid_plot(KID_SCORES_cgan_acgan,"CGAN-ACGAN")